# Movie Recommendation Engine

In [146]:
import pandas as pd
import os


In [147]:
df1=pd.read_csv("buses.csv")
df1.head()

,busId,busname,place
0,1,mahanagar,ringroad
1,2,mahanagar,ringroad
2,3,mahanagar,ringroad
3,4,mahanagar,ringroad
4,5,mahanagar,ringroad


In [148]:
df2=pd.read_csv("ratings.csv")
df2.head()

,passengerId,busId,busrating,timestamp
0,1,1,4.0,123456
1,2,3,4.0,123456
2,3,6,4.0,123456
3,4,7,5.0,123456
4,5,8,5.0,123456


In [149]:

df3=pd.merge(df1,df2).drop(['place','timestamp'],axis=1)
df3.head()

,busId,busname,passengerId,busrating
0,1,mahanagar,1,4.0
1,2,mahanagar,56,3.0
2,2,mahanagar,45,4.0
3,2,mahanagar,44,4.0
4,2,mahanagar,56,4.0


In [151]:
passengerRatings = df3.pivot_table(index=['passengerId'],columns=['busname'],values='busrating')
passengerRatings.head()
print("Before: ",passengerRatings.shape)
passengerRatings = passengerRatings.dropna(thresh=5, axis=1).fillna(0,axis=1)

#passengerRatings.fillna(0, inplace=True)
print("After: ",passengerRatings.shape)
passengerRatings.head()

Before:  (58, 3)
After:  (58, 3)


busname,mahanagar,mayur,nepal yatayat
passengerId,,,
1,4.0,0.0,0.0
2,4.0,0.0,5.0
3,4.0,0.0,0.0
4,5.0,0.0,0.0
5,5.0,0.0,4.0


In [152]:
corrMatrix = passengerRatings.corr(method='pearson')
corrMatrix

busname,mahanagar,mayur,nepal yatayat
busname,,,
mahanagar,1.000000,-0.369023,-0.048990
mayur,-0.369023,1.000000,-0.299796
nepal yatayat,-0.048990,-0.299796,1.000000


In [153]:
for i in corrMatrix.index:
    print(i)

mahanagar
mayur
nepal yatayat


In [154]:
def get_similar(bus_name,rating):
    similar_ratings = corrMatrix[bus_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [155]:
buses = [("mahanagar",5),("nepal yatayat",3),("mayur",1)]
similar_buses = pd.DataFrame()
for buses,busrating in buses:
    similar_buses = similar_buses.append(get_similar(buses,busrating),ignore_index = True)


C:\Users\dell\AppData\Local\Temp\ipykernel_10480\3825561084.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_buses = similar_buses.append(get_similar(buses,busrating),ignore_index = True)
C:\Users\dell\AppData\Local\Temp\ipykernel_10480\3825561084.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_buses = similar_buses.append(get_similar(buses,busrating),ignore_index = True)
C:\Users\dell\AppData\Local\Temp\ipykernel_10480\3825561084.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_buses = similar_buses.append(get_similar(buses,busrating),ignore_index = True)


In [156]:
similar_buses.sum().sort_values(ascending=False).head(10)

busname
mahanagar        3.029040
nepal yatayat    0.827220
mayur           -2.572456
dtype: float64

In [157]:
import pickle
with open("buses_pickle","wb") as f:
    pickle.dump(corrMatrix,f)

In [158]:
with open("buses_pickle","rb") as f:
    model=pickle.load(f)

In [159]:
corrMatrix.index[1]

'mayur'

In [160]:
model.index[2]

'nepal yatayat'

In [163]:
__file__='buses_pickle'
filename=os.path.dirname(os.path.abspath(__file__))+'/buses_pickle'